## Dane makroekonomiczne – rozszerzona weryfikacja prognoz
Usprawnienia obejmują walidację wejścia, lepszą agregację, nowe metryki oraz dodatkowe testy statystyczne i wizualizacje.


In [ ]:
from pathlib import Path
from collections import Counter
import json, re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import mstats


In [ ]:
# Stały katalog bazowy i walidacja wejścia
BASE_DIR = Path('wyniki LLM')
FILES = {
    'bankier': 'bankier_llm_extracted.csv',
    'business insider': 'businessinsider_llm_extracted.csv',
    'money': 'money_llm_extracted.csv',
    'obserwator finansowy': 'obserwatorfinansowy_llm_extracted_clean.csv',
    'pap': 'pap_llm_extracted.csv',
    'tvn24': 'tvn24_llm_extracted.csv',
}

rows = []
data_frames = []
for name, fname in FILES.items():
    path = BASE_DIR / fname
    if not path.exists():
        rows.append({'source': name, 'articles': np.nan, 'note': f'missing: {path}'})
        continue
    try:
        df_src = pd.read_csv(path)
    except Exception as e:
        rows.append({'source': name, 'articles': np.nan, 'note': f'ERROR: {e}'})
        continue
    rows.append({'source': name, 'articles': len(df_src), 'note': 'ok'})
    df_src['news_source'] = name
    data_frames.append(df_src)

articles_per_source = pd.DataFrame(rows)
display(articles_per_source)

sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6))
sns.barplot(data=articles_per_source, x='source', y='articles', palette='viridis', hue='note', dodge=False, legend=False)
plt.title('Articles per Source (validated)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

if not data_frames:
    raise SystemExit('Brak poprawnie wczytanych plików z BASE_DIR')

df = pd.concat(data_frames, ignore_index=True)


### Walidacja i normalizacja prognoz
Liczba odrzuconych rekordów jest raportowana według przyczyny oraz liczba zachowanych prognoz na zmienną/rok.


In [ ]:
# Parse forecasts_json -> normalized long df (one row per variable/year)
valid_vars = {'gdp','inflation','unemployment','deficit','public_debt','interest_rate','fx','wages','other'}
parse_stats = Counter()
parsed_rows = []

def _parse_row(raw: str, stats_counter: Counter):
    try:
        items = json.loads(raw)
    except Exception:
        stats_counter['json_error'] += 1
        return []
    out = []
    for f in items:
        stats_counter['items_total'] += 1
        if not isinstance(f, dict) or 'variable' not in f or 'value' not in f:
            stats_counter['invalid_shape'] += 1
            continue
        var = str(f.get('variable', '')).strip().lower()
        if var not in valid_vars:
            stats_counter['unknown_variable'] += 1
            continue
        horizon = str(f.get('horizon', '')).strip()
        m = re.search(r'\b(\d{4})\b', horizon)
        if not m:
            stats_counter['missing_year'] += 1
            continue
        year = int(m.group(1))
        val = pd.to_numeric(str(f.get('value', '')).replace(',', '.'), errors='coerce')
        if pd.isna(val):
            stats_counter['non_numeric'] += 1
            continue
        stats_counter[f'kept_{var}'] += 1
        out.append({'variable': var, 'year': year, 'value': float(val)})
    return out

for _, r in df.iterrows():
    parse_stats['rows_total'] += 1
    recs = _parse_row(r.get('forecasts_json', '[]'), parse_stats)
    parse_stats['rows_with_forecast'] += int(len(recs) > 0)
    for rr in recs:
        rr.update({
            'url': r.get('url'),
            'who': r.get('main_topic') or r.get('title_search') or '',
            'news_source': r.get('news_source'),
        })
        parsed_rows.append(rr)

forecasts_long = pd.DataFrame(parsed_rows)

print('=== Statystyki walidacji ===')
for k in sorted(parse_stats):
    print(f"{k}: {parse_stats[k]}")

missing_by_var_year = (forecasts_long.groupby(['variable', 'year']).size()
                       .rename('n')
                       .reset_index())
print('
Liczba prognoz na (zmienna, rok):')
display(missing_by_var_year.head(20))


### Agregacja prognoz i kontrola duplikatów
Średnia, mediana i odchylenie standardowe dla wielu prognoz tego samego autora/roku.


In [ ]:
key_vars = ['gdp', 'public_debt', 'deficit', 'inflation', 'unemployment', 'interest_rate']
forecast_stats = (
    forecasts_long[forecasts_long['variable'].isin(key_vars)]
    .groupby(['year', 'who', 'news_source', 'variable'])
    .agg(
        forecast_mean=('value', 'mean'),
        forecast_median=('value', 'median'),
        forecast_std=('value', 'std'),
        n=('value', 'count'),
    )
    .reset_index()
)

wide_mean = forecast_stats.pivot(index=['year', 'who'], columns='variable', values='forecast_mean').reset_index()
wide_median = forecast_stats.pivot(index=['year', 'who'], columns='variable', values='forecast_median').reset_index()

print('Przykładowe wiersze (średnie prognozy):')
display(wide_mean.head())


### Łączenie z wartościami rzeczywistymi i raport braków


In [ ]:
with open('dane_makroekonomiczne.json', 'r', encoding='utf-8') as f:
    actual_raw = json.load(f)

name_map = {
    'gdp': 'Realny wzrost PKB (% r/r)',
    'inflation': 'Inflacja CPI (średnioroczna, %)',
    'interest_rate': 'Stopa referencyjna NBP (średnioroczna, %)',
    'unemployment': 'Stopa bezrobocia (BAEL / Eurostat, %)',
    'deficit': 'Deficyt sektora GG (% PKB)',
    'public_debt': 'Dług publiczny (GG, % PKB)',
}

ind_by_name = {ind['name']: ind for ind in actual_raw.get('indicators', [])}
actual_rows = []
for var, ind_name in name_map.items():
    ind = ind_by_name.get(ind_name)
    if not ind:
        continue
    for y, vals in ind.get('values', {}).items():
        try:
            year = int(y)
        except Exception:
            continue
        val = vals.get('eurostat_gus')
        if val is None:
            val = vals.get('oryginalne')
        if val is None:
            continue
        actual_rows.append({'variable': var, 'year': year, 'actual': float(val)})

actual_df = pd.DataFrame(actual_rows)

comparison = forecast_stats.merge(actual_df, on=['variable', 'year'], how='left')
missing_actual_share = comparison['actual'].isna().mean()
print(f'Brakujące wartości actual: {missing_actual_share:.1%} z {len(comparison)} wierszy')
print('Rozkład braków per zmienna:')
display(comparison.groupby('variable')['actual'].apply(lambda s: s.isna().mean()).to_frame('missing_share'))


### Metryki jakości prognoz (MAE, MAPE, sMAPE, RMSE) z winsoryzacją APE


In [ ]:
# Filtr obserwacji z rzeczywistą wartością
comparison_valid = comparison[comparison['actual'].notna()].copy()

# Obliczenia błędów
comparison_valid['error'] = comparison_valid['forecast_mean'] - comparison_valid['actual']
comparison_valid['abs_error'] = comparison_valid['error'].abs()
comparison_valid['pct_error'] = np.where(
    comparison_valid['actual'].abs() > 1e-9,
    comparison_valid['error'] / comparison_valid['actual'] * 100,
    np.nan,
)
comparison_valid['abs_pct_error'] = comparison_valid['pct_error'].abs()

# Winsoryzacja skrajnych wartości procentowych
comparison_valid['abs_pct_error_w'] = mstats.winsorize(comparison_valid['abs_pct_error'], limits=[0, 0.05])
comparison_valid['smape'] = 200 * np.abs(comparison_valid['forecast_mean'] - comparison_valid['actual']) / (
    np.abs(comparison_valid['forecast_mean']) + np.abs(comparison_valid['actual'])
)

rmse_by_var = comparison_valid.groupby('variable')['error'].apply(lambda s: np.sqrt(np.mean(np.square(s)))).rename('RMSE')

summary_by_var = (
    comparison_valid.groupby('variable').agg(
        Mean_Error=('error', 'mean'),
        Std_Error=('error', 'std'),
        MAE=('abs_error', 'mean'),
        Median_AE=('abs_error', 'median'),
        MAPE_pct=('abs_pct_error', 'mean'),
        MAPE_winsor_pct=('abs_pct_error_w', 'mean'),
        sMAPE=('smape', 'mean'),
        N=('error', 'count'),
    )
).join(rmse_by_var).round(3)
print('\n=== Błędy prognoz według zmiennych ===')
display(summary_by_var)


summary_by_year = (
    comparison_valid.groupby('year').agg(
        Mean_Error=('error', 'mean'),
        Std_Error=('error', 'std'),
        MAE=('abs_error', 'mean'),
        MAPE_pct=('abs_pct_error', 'mean'),
        sMAPE=('smape', 'mean'),
        N=('error', 'count'),
    ).round(3)
)
print('\n=== Błędy prognoz według roku ===')
display(summary_by_year)


### Testy statystyczne porównujące źródła/roczniki


In [ ]:
# Porównanie błędów między źródłami (t-test lub Wilcoxon w wersji rank-sum)
source_stats = comparison_valid.groupby('news_source')['error'].agg(['count', 'mean'])
print('Błędy per źródło:')
display(source_stats)

sources = [s for s, n in source_stats['count'].items() if n >= 5]
if len(sources) >= 2:
    s1, s2 = sources[:2]
    e1 = comparison_valid.loc[comparison_valid['news_source'] == s1, 'error']
    e2 = comparison_valid.loc[comparison_valid['news_source'] == s2, 'error']
    t_res = stats.ttest_ind(e1, e2, equal_var=False, nan_policy='omit')
    u_res = stats.ranksums(e1, e2)
    print(f"
T-test ({s1} vs {s2}): stat={t_res.statistic:.3f}, p={t_res.pvalue:.4f}")
    print(f"Rank-sum ({s1} vs {s2}): stat={u_res.statistic:.3f}, p={u_res.pvalue:.4f}")
else:
    print('Za mało danych na testy między źródłami (wymagane >=5 obserwacji na źródło).')

# Trend błędu w czasie (regresja liniowa)
lin_res = stats.linregress(comparison_valid['year'], comparison_valid['error'])
print(f"
Trend błędu w czasie: slope={lin_res.slope:.4f}, p-value={lin_res.pvalue:.4f}, r^2={lin_res.rvalue**2:.3f}")

# Test KS porównujący rozkłady błędów dla dwóch największych źródeł
source_sizes = comparison_valid['news_source'].value_counts()
if len(source_sizes) >= 2:
    s1, s2 = source_sizes.index[:2]
    ks_res = stats.ks_2samp(
        comparison_valid.loc[comparison_valid['news_source'] == s1, 'error'],
        comparison_valid.loc[comparison_valid['news_source'] == s2, 'error'],
        alternative='two-sided',
    )
    print(f"KS-test ({s1} vs {s2}): stat={ks_res.statistic:.3f}, p={ks_res.pvalue:.4f}")
else:
    print('Za mało źródeł na test KS.')


### Wizualizacje: boxplot/violin, KDE i heatmapy


In [ ]:
sns.set_palette('tab10')

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.boxplot(data=comparison_valid, x='variable', y='error', ax=axes[0])
axes[0].axhline(0, color='red', linestyle='--', linewidth=1)
axes[0].set_title('Boxplot błędów per zmienna')
axes[0].tick_params(axis='x', rotation=45)

sns.violinplot(data=comparison_valid, x='year', y='error', inner='quartile', ax=axes[1])
axes[1].axhline(0, color='red', linestyle='--', linewidth=1)
axes[1].set_title('Violin plot błędów per rok')
plt.tight_layout()
plt.show()

# KDE błędów
plt.figure(figsize=(10, 6))
for var, grp in comparison_valid.groupby('variable'):
    sns.kdeplot(grp['error'], label=var, fill=False)
plt.axvline(0, color='black', linestyle='--')
plt.title('Gęstości błędów prognoz')
plt.legend()
plt.tight_layout()
plt.show()

# Heatmapa MAPE per (zmienna, rok)
heatmap_data = comparison_valid.pivot_table(index='variable', columns='year', values='abs_pct_error', aggfunc='mean')
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='mako', cbar_kws={'label': 'MAPE (%)'})
plt.title('MAPE średnie wg zmiennej i roku')
plt.tight_layout()
plt.show()


---
Powyższe komórki dodają walidację wejścia, pełniejszą agregację, dodatkowe metryki (RMSE, sMAPE), testy statystyczne oraz nowe wizualizacje.
